# **Библиотека Dash**

**Что это за библиотека и что позволяет делать?**
Dash — библиотека для языка Python с открытым исходным кодом, предназначенная для создания *реактивных* веб-приложений.

Она упрощает создание графических пользовательских интерфейсов (GUI) для анализа данных. Dash позволяет легко создавать и делиться результатами анализа данных с помощью интерактивных дашбордов, используя *только код Python*. (на самом деле чтобы было стилево все таки нужно знание HTML и CSS)

### **Импорты**

Импортируем все, что может нам понадобится. Из важного dcc и html. В dcc мы импортируем функции, которые позволяют создавать различные элементы нашего приложения. А в html находятся функции, которые позволяют создавать HTML элементы. 


In [4]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### ***Hello, world!***
Давайте создадим простое приложение, которое будет выводить "Hello, world!".    

Вообще, приложения Dash состоят из нескольких частей: app, layout, callback.

layout - это структура приложения, которая определяет, какие элементы будут отображаться в приложении и как они будут расположены. Стиль этих элементов определяется с помощью HTML и CSS. Я не шарю в HTML и CSS, поэтому не буду подробно расписывать, как именно они там выглядят. А CSS стиля здесь не будет вообще, короче к сожалению я не эстетик в этом плане, но по функционалу все будет

callback - это функция, которая будет вызываться при изменении какого-либо элемента нашего приложения. И вообще она позволяет нам управлять поведением нашего приложения и в принципе работать с данными.

app - это объект, который в принципе создает наше приложение. Очевидно, что без него не будет работать ничего.

In [3]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Hello, world!')
])

if __name__ == '__main__':
    app.run(debug=True)

У нас все получилось! Давайте теперь загрузим данные и создадим приложение, которое будет выводить графики на основе этих данных.

### **Загрузка данных**

Я постаралась выбрать интересные данные, которые помогут нам понять, как именно работает Dash. Их я взяла с Kaggle, если это важно.

In [5]:
data = pd.read_csv("wine_data.csv")
pd.set_option('display.max_columns', 16)
print(data.columns)
print(data)

Index(['level_0', 'index', 'country', 'description', 'rating', 'price',
       'province', 'title', 'variety', 'winery', 'color', 'countryID',
       'varietyID', 'colorID', 'provinceID', 'wineryID'],
      dtype='object')
        level_0  index   country  \
0             0      0  Portugal   
1             1   1366  Portugal   
2             2   4018  Portugal   
3             3  18808  Portugal   
4             4  56322  Portugal   
...         ...    ...       ...   
100223   100223  69483        US   
100224   100224  69493        US   
100225   100225  81234        US   
100226   100226  85115        US   
100227   100227  43274        US   

                                              description  rating  price  \
0       This is ripe and fruity, a wine that is smooth...      87   15.0   
1       This wine is light in tannins and ripe in frui...      85   11.0   
2       Towards the western end of the Douro vineyards...      87   15.0   
3       This Avidagos vineyard in the Co


### **Создание приложения**


Начнем с **простого приложения**, которое будет выводить базовые графики. 


#### **App и определение графиков**
Здесь мы создаем три графика:
1. График, который показывает соотношение цены и рейтинга вин
2. График, который показывает распределение рейтингов по странам.
3. Круговая диаграмма, которая показывает распределение вин по цвету.

Графики задаются с помощью функций типа px.scatter, px.box и px.pie. Есть и другие.

В чем отличия функций? 
px.scatter используется для создания точечного графика, px.box используется для создания сложного графика, который показывает распределение данных по какому-то признаку, px.pie используется для создания круговой диаграммы. 
Как выбрать подходящую? 
На самом деле, кажется, что только смотреть каждый раз в документации, потому что их много.

   
Теперь перейдем к layout.




In [47]:
# Создаем приложение Dash
app = dash.Dash(__name__)

# Создаем несколько базовых графиков

#1. График, который показывает соотношение цены и рейтинга вин
fig_price_rating = px.scatter(data, 
    x='price',   # что по оси X
    y='rating',   # что по оси Y
    color='color',  # цвет точки зависит от цвета вина
    hover_data=['title', 'country'],   # показывает название вина и страну при наведении
    title='Соотношение цены и рейтинга вин' #тайтл графика
)

#2. График, который показывает распределение рейтингов по странам
fig_country_rating = px.box(data, 
    x='country',   # что по оси X
    y='rating',  # что по оси Y 
    title='Распределение рейтингов по странам'  # тайтл графика
    
)

#3. Круговая диаграмма распределения цветов вин
fig_pie = px.pie(data, 
    names='color',             # группировка по цвету вина
    title='Распределение вин по цвету'    # тайтл графика
)

#### **Layout**

**Что здесь происходит?**

Мы создаем layout, который состоит из нескольких частей:
1. Заголовок
2. Секция с фильтрами
3. Секция с графиками

**Для чего нужны фильтры?**

Фильтры нужны для того, чтобы мы могли фильтровать данные в нашем приложении. Например, мы можем выбрать определенную страну, цвет вина или цену.
У нас есть три фильтра:
1. Выбор стран.
Реализован как обычный выпадающий список.       
2. Выбор цены
Реализован в виде range slider. Здесь мы можем определить диапазон цен, которые будут отображаться на графике с помощью параметров min, max. Также мы можем указать шаг с помощью параметра step и отметки с помощью параметра marks.   
3. Выбор цвета  
Реализован как обычный выпадающий список, но с возможностью выбора нескольких цветов. Это опредляется параметром multi = True. 

Как они вообще реализованы? 

В функции dcc.Dropdown мы можем указать options, которые будут являться выбором для нашего фильтра. И базовый для html placeholder, который будет отображаться в поле выбора.

**Что расположено в секции с графиками?**

В секции с графиками расположены два графика, которые мы создали ранее. Они отображаются с помощью функции dcc.Graph. В нее нам нужно передать id графика и figure, которое мы создали ранее.   

Чтобы все это нажималось и работало, нам нужно создать callback. Рассмотрим в следующем блоке кода.


In [48]:
app.layout = html.Div([
    # Заголовок приложения
    html.H1('Анализ данных о винах', style={'textAlign': 'center'}),
    
    html.Div([
        # Секция с фильтрами
        html.Div([
            html.H3('Фильтры:'),
            #Выпадающий список для выбора страны
            dcc.Dropdown(
                id='country-filter',
                options=[{'label': country, 'value': country} 
                         for country in data['country'].unique()],  #создаем список опций из уникальных стран
                placeholder='Выберите страну'  #подпись в поле выбора
            ),
            # слайдер для выбора диапазона цен
            dcc.RangeSlider(
                id='price-range',
                min=data['price'].min(),  
                max=data['price'].max(), 
                step=10, 
                marks={i: f'${i}' for i in range(0, int(data['price'].max()), 200)},  # Отметки на слайдере каждые $200
                value=[data['price'].min(), data['price'].max()]  #начальный диапазон это все цены
            ),
            #выпадающий список для выбора цвета вина
            dcc.Dropdown(
                id='color-filter',
                options=[{'label': color, 'value': color} 
                         for color in data['color'].unique()],  #опции на выбор это цвета
                placeholder='Выберите цвет вина', #подпись в поле выбора
                multi=True  #возможность выбрать несколько цветов(крутая!!!)
            )
        ]),
        
        #секция с графиками, здесб просто перечисляем графики, которые объявляли ранее и даем им id, чтобы они отображались на страничке
        html.Div([
            dcc.Graph(id='price-rating-graph', figure=fig_price_rating), 
            dcc.Graph(id='country-rating-graph', figure=fig_country_rating),
            dcc.Graph(id='pie-chart', figure=fig_pie)
        ])
    ])
])

#### **Callback**
В callback мы передаем два параметра:
1. Output - это то, что мы хотим получить на выходе(сами графики)
2. Input - это то, что мы даем на входе(фильтры)

Дальше пишем функцию, которая будет принимать наши фильтры и возвращать наши графики.  
Отбираем данные, согласно выбранным фильтрам и создаем заявленные графики.  
Чтобы создать графики мы буквально передаем в функции px.scatter px.box px.pie   те данные, которые мы объявляли в самом начале и отфильтрованные согласно выбранным фильтрам.  



In [49]:
@app.callback(
    # Определяем выходные данные(графики)
    [Output('price-rating-graph', 'figure'),   
     Output('country-rating-graph', 'figure'),  
     Output('pie-chart', 'figure')]
    # Определяем входные данные:(фильтры)
    [Input('country-filter', 'value'),  
     Input('price-range', 'value'),             
     Input('color-filter', 'value')]            
)
def update_graphs(selected_country, price_range, selected_colors):
    filtered_data = data.copy()
    
    
    if selected_country:
        filtered_data = filtered_data[filtered_data['country'] == selected_country] #фильтруем по выбранной стране
    
    
    filtered_data = filtered_data[
        (filtered_data['price'] >= price_range[0]) &  # цена больше или равна минимальной выбранной
        (filtered_data['price'] <= price_range[1])    #меньше максимальной выбранной
    ]
    
    
    if selected_colors:  #фильтруем по выбранным цветам
        filtered_data = filtered_data[filtered_data['color'].isin(selected_colors)]
    
    # Создаем обновленные графики, с теми же полями, что были раньше, но передаем отфильтрованные данные
    fig_price_rating = px.scatter(filtered_data, 
        x='price',                              
        y='rating',                              
        color='color',                          
        hover_data=['title', 'country'], 
        title='Соотношение цены и рейтинга вин'
    )

    fig_country_rating = px.box(filtered_data, 
        x='country',      
        y='rating', 
        title='Распределение рейтингов по странам' 
    )

    fig_pie = px.pie(filtered_data, 
        names='color', 
        title='Распределение вин по цвету'
    )

    return fig_price_rating, fig_country_rating, fig_pie

#### **Запуск через 3, 2, 1...**
Давайте попробуем запустить приложение. Его можно запускать в принципе прям в ноутбуке, но я предпочитаю запускать в отдельном окне.


In [50]:
import webbrowser
from threading import Timer

def open_browser():
    webbrowser.open('http://127.0.0.1:8050/')

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)

Видим, что приложение запустилось и даже работает!
Теперь предлагаю посмотреть какие еще виды графиков можно создать с помощью Dash. На самом деле их очень много, поэтому я выбрала те, которые показались мне наиболее интересными. Их много потому что Dash позволяет создавать графики с помощью библиотеки Plotly. В сумме их около 300)) и они собраны в несколько категорий и подходят для разных сфер.



#### **App и графики**
Создадим график в виде карты мира, который будет показывать средний рейтинг вин по странам. 
Замечу, что теперь нам нужно сначала сделать запрос к данным, чтобы получить средний рейтинг вин по странам. Мы делаем это с помощью sql запроса. Далее мы создаем график с помощью функции go.Figure. В нее мы передаем данные, которые мы получили из запроса и параметры, которые определяют как будет выглядеть график. Еще мы дописали функцию update_layout, которая позволяет нам настроить внешний вид графика. 

Следующий график - это график в виде столбцов, который показывает топ сортов вин по рейтингу. Мы создаем график с помощью функции px.bar и передаем в нее данные, которые мы получили из SQL запроса.

Третий график - он не совсем подходит для моих данных, но он довольно полезный в реальных задачах. Он показывает соотношение цены и рейтинга вин, но в данном случае размер точки зависит от количества отзывов(зачем?). Цвет точки зависит от страны. 

Четвертый график - это sunburst диаграмма, которая показывает иерархию вин: страна -> цвет -> сорт. Размер сегментов зависит от цены, цвет зависит от рейтинга. В интернете сказали, что этот график реально полезен при анализе данных(ок)

Последний график - 3D визуализация, которая показывает топ виноделен, цены и рейтинги. Невероятно!

Пока делала все эти примеры, кажется, что стало понятно, что выбранные мною данные не оч интересны для обзора такими графиками, но мне кажется, что графики все равно информативные!


In [ ]:
reviews_count = data.groupby('title')['rating'].count().reset_index() #добавим в таблицу столбек с кол-вом отзывов
reviews_count.columns = ['title', 'n_reviews']                          # он нужен для четвертого графика
data = data.merge(reviews_count, on='title', how='left')

In [68]:
# Создаем новые графики

# 1. Карта мира с средним рейтингом вин по странам
# Считаем средний рейтинг для каждой страны
avg_ratings = data.groupby('country')['rating'].mean().reset_index()

# Создаем карту мира
fig_world_map = go.Figure(data=go.Choropleth(
    locations=data['country'].unique(),     # передаесписок уникальных стран
    locationmode='country names',     # определяем страны по названиям
    z=avg_ratings['rating'],             # значения чтобы шкала цветная была
    text=avg_ratings['country'],          # текст при наведении
    colorscale='Viridis',                   # цветовая схема
    colorbar_title="Средний рейтинг"    # тайтл цветовой шкалы
))
# Настраиваем внешний вид карты
fig_world_map.update_layout(
    title='Средний рейтинг вин по странам',
    geo=dict(showframe=False),              # убираем рамку карты
)

# 3. Топ сортов вин по рейтингу
# Группируем данные по сортам вина, считаем среднее и количество
top_varieties = data.groupby('variety')['rating'].agg(['mean', 'count'])\
    .sort_values('mean', ascending=False)\
    .head(50)\
    .reset_index()   #сортируем по среднему рейтингу, берем топ-50, сбрасываем индекс

# Создаем столбчатую диаграмму
fig_top_varieties = px.bar(top_varieties, 
    x='variety',   # чтопо оси X
    y='mean',  # что по оси Y
    title='Топ сортов вин по рейтингу',
    labels={'mean': 'Средний рейтинг',    # подписи осей
            'variety': 'Сорт вина'}
)

# 4. Отношение цены и рейтинга вин  
fig_price_rating = px.scatter(data, 
    x='price',  
    y='rating',                         
    size='n_reviews',   # размер точки = количество отзывов(кринж)
    color='country',   # цвет точки это страны
    hover_name='title',  # название вина при наведении(будет как заголовок в инф блоке)
    hover_data=['color', 'variety', 'n_reviews'],  # ще текст при наведении(просто текст в инф блоке)
    log_x=True,    #логарифмическая шкала для цены(чтобы не было огромных точек)
    size_max=60,     # максимальный размер точки(чтобы не было огромных точек)
    title='Соотношение цены и рейтинга вин (размер = количество отзывов)'
)
# подписи осей
fig_price_rating.update_layout(
    xaxis_title="Цена",
    yaxis_title="Рейтинг"
)

# 5.sunburst диаграмма
fig_sunburst = px.sunburst(
    data, 
    path=['country', 'color', 'variety'],  # Иерархия
    values='price', #Размер сегментов
    color='rating',  # От чего зависи цвет
    color_continuous_scale='RdYlGn',  #цветовая схема
    title='Иерархия вин: страна -> цвет -> сорт (размер = цена, цвет = рейтинг)'
)

fig_sunburst.update_layout(
    width=800,
    height=800
);

top_wineries = data.groupby('winery').size()\
    .sort_values(ascending=False)\
    .head(20).index
winery_data = data[data['winery'].isin(top_wineries)]

fig_3d = px.scatter_3d(winery_data, 
    x='winery', 
    y='price', 
    z='rating', 
    color='color',  # цвет точек по типу вина
    size='n_reviews',  #размер точки это количество отзывов
    hover_name='title', 
    hover_data=['country', 'variety'],  
    title='3D визуализация: топ виноделен, цены и рейтинги',
    labels={                               # подписи осей
        'winery': 'Винодельня',
        'price': 'Цена',
        'rating': 'Рейтинг',
        'color': 'Цвет вина'
    }
)

fig_3d.update_layout(
    scene = dict(
        xaxis_title='Винодельня',
        yaxis_title='Цена',
        zaxis_title='Рейтинг'
    ),
    width=1000,   # ширина графика
    height=800,     #высота графика
    scene_camera=dict( #астройка начального угла обзора
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=2, y=2, z=1.5)
    )
);

#### **Layout**

Размещение фильтров оставляем прежним, как и секция с графиками.


In [69]:
# Определяем layout приложения
app.layout = html.Div([
    html.H1('Анализ данных о винах', style={'textAlign': 'center'}),
    
    # Секция с фильтрами
    html.Div([
        html.Div([
            html.H3('Фильтры:'),
            dcc.Dropdown(
                id='country-filter',
                options=[{'label': country, 'value': country} 
                         for country in data['country'].unique()],
                placeholder='Выберите страну'
            ),
            dcc.RangeSlider(
                id='price-range',
                min=data['price'].min(),
                max=data['price'].max(),
                step=10,
                marks={i: f'${i}' for i in range(0, int(data['price'].max()), 200)},
                value=[data['price'].min(), data['price'].max()]
            ),
            html.Br(),
            dcc.Dropdown(
                id='color-filter',
                options=[{'label': color, 'value': color} 
                         for color in data['color'].unique()],
                placeholder='Выберите цвет вина',
                multi=True
            ),
        ]),
        
        # Секция с графиками
        html.Div([
                dcc.Graph(id='world-map', figure=fig_world_map),
                dcc.Graph(id='top-varieties', figure=fig_top_varieties),
                dcc.Graph(id='price-rating-graph', figure=fig_price_rating),
                dcc.Graph(id='sunburst-graph', figure=fig_sunburst),
                dcc.Graph(id='3d-graph', figure=fig_3d)
            ]
        )
    ])
])


#### **Callback**

Теперь мы переопределяем графики с учетом отфильтрованных данных.


In [70]:
# Обновляем callback
@app.callback(
    [Output('world-map', 'figure'),
     Output('top-varieties', 'figure'),
     Output('price-rating-graph', 'figure'),
     Output('sunburst-graph', 'figure'),
     Output('3d-graph', 'figure')
     ],
    [Input('country-filter', 'value'),
     Input('price-range', 'value'),
     Input('color-filter', 'value')]
)
def update_graphs(selected_country, price_range, selected_colors):
    filtered_data = data.copy()
    
    if selected_country:
        filtered_data = filtered_data[filtered_data['country'] == selected_country]
    
    if selected_colors:
        filtered_data = filtered_data[filtered_data['color'].isin(selected_colors)]
    
    filtered_data = filtered_data[
        (filtered_data['price'] >= price_range[0]) & 
        (filtered_data['price'] <= price_range[1])
    ]
    
    #обновляем все графики с отфильтрованными данными
    avg_ratings = filtered_data.groupby('country')['rating'].mean().reset_index()
    fig_world_map = go.Figure(data=go.Choropleth(
        locations=filtered_data['country'].unique(),
        locationmode='country names',
        z=avg_ratings['rating'],
        text=avg_ratings['country'],
        colorscale='Viridis',
        colorbar_title="Средний рейтинг"
    ))
    fig_world_map.update_layout(
        title='Средний рейтинг вин по странам',
        geo=dict(showframe=False),
    )
    
    
    top_varieties = filtered_data.groupby('variety')['rating'].agg(['mean', 'count'])\
        .sort_values('mean', ascending=False)\
        .head(50)\
        .reset_index()
    
    fig_top_varieties = px.bar(top_varieties, 
        x='variety', 
        y='mean',
        title='Топ сортов вин по рейтингу',
        labels={'mean': 'Средний рейтинг', 'variety': 'Сорт вина'}
    )

    fig_price_rating = px.scatter(filtered_data, 
    x='price', 
    y='rating',
    size='n_reviews',  
    color='country',
    hover_name='title',
    hover_data=['color', 'variety', 'n_reviews'],  
    log_x=True,
    size_max=60,
    title='Соотношение цены и рейтинга вин (размер = количество отзывов)'
    )

    fig_sunburst = px.sunburst(
        filtered_data, 
        path=['country', 'color', 'variety'],
        values='price',
        color='rating',
        color_continuous_scale='RdYlGn',
        title='Иерархия вин: страна -> цвет -> сорт (размер = цена, цвет = рейтинг)'
    )

    top_wineries = filtered_data.groupby('winery').size()\
        .sort_values(ascending=False)\
        .head(20).index
    winery_data = filtered_data[filtered_data['winery'].isin(top_wineries)]

    fig_3d = px.scatter_3d(winery_data, 
        x='winery',
        y='price',
        z='rating',
        color='color',
        size='n_reviews',
        hover_name='title',
        hover_data=['country', 'variety'],
        title='3D визуализация: топ виноделен, цены и рейтинги',
        labels={
            'winery': 'Винодельня',
            'price': 'Цена',
            'rating': 'Рейтинг',
            'color': 'Цвет вина'
        }
    )
    
    
    return  fig_world_map,  fig_top_varieties, fig_price_rating, fig_sunburst, fig_3d

Запустим и посмотрим, что у нас получилось

In [71]:
import webbrowser
from threading import Timer

def open_browser():
    webbrowser.open('http://127.0.0.1:8050/')

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

На этом все! Мы рассмотрели основные компоненты Dash и создали несколько приложений. Библиотека очень обширная, по большей части из-за доступности гарфиков из plotly. Она ее дополняет и позволяет создавать веб-приложения для анализа данных, причем эти приложения интерактивные и фильтров/полей тоже очень много. 
# **Спасибо за внимание!**